### Importing Libraries

In [1]:
import pandas as pd
import requests
import json
from IPython.display import JSON



In [16]:
from api_keys import gans_aws_key as awskey
from api_keys import mysql_key as sql_key


### Read cities.csv file to get city information

In [3]:
cities_df = pd.read_csv('cities.csv')
print(cities_df.info())
cities_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   city        4 non-null      object 
 1   country     4 non-null      object 
 2   latitude    4 non-null      float64
 3   longitude   4 non-null      float64
 4   population  4 non-null      int64  
 5   city_id     4 non-null      int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 324.0+ bytes
None


,city,country,latitude,longitude,population,city_id
0,Berlin,Germany,52.3112,13.2418,3850809,1
1,London,United Kingdom,51.3026,0.7390,8799800,2
2,Manchester,United Kingdom,53.2846,2.1443,551938,3
3,Istanbul,Turkey,41.0049,28.5718,15907951,4


### Create a city_id that will be used as a unique identifier/ Primary key in SQL


In [4]:
cities_df['city_id'] = cities_df.index
cities_df

,city,country,latitude,longitude,population,city_id
0,Berlin,Germany,52.3112,13.2418,3850809,0
1,London,United Kingdom,51.3026,0.7390,8799800,1
2,Manchester,United Kingdom,53.2846,2.1443,551938,2
3,Istanbul,Turkey,41.0049,28.5718,15907951,3


### Get data from cities 

In [5]:
from api_keys import my_api_key as wkey

In [6]:
#Berlin
cities_df.loc[0, 'latitude']

52.3112

In [7]:
city_conditions_dict = {
                        'city_id': [],
                        'time' : [],
                        'temp_min': [],
                        'temp_max': [],
                        'pop': [],
                        'wind_speed': [],
                        'description': [],
                        'humidity': []}



for i, city in enumerate(cities_df['city_id']):
    #city_name = cities_df.iloc[i]['city']
    forecast_url = f"http://api.openweathermap.org/data/2.5/forecast?q={cities_df.iloc[i]['city']}&appid={wkey}&units=metric"
    response = requests.get(forecast_url)
    weather_json = response.json()
    
    for j in weather_json['list']:       
        city_conditions_dict['city_id'].append(city)
        city_conditions_dict['time'].append(j['dt_txt'])
        city_conditions_dict['temp_min'].append(j['main']['temp_min'])
        city_conditions_dict['temp_max'].append(j['main']['temp_max'])
        city_conditions_dict['wind_speed'].append(j['wind']['speed'])
        city_conditions_dict['humidity'].append(j['main']['humidity'])
        city_conditions_dict['pop'].append(j['pop'])
        city_conditions_dict['description'].append(j['weather'][0]['description'])


In [8]:
response.json()

{'cod': '200',
 'message': 0,
 'cnt': 40,
 'list': [{'dt': 1697371200,
   'main': {'temp': 20.23,
    'feels_like': 20.14,
    'temp_min': 20.23,
    'temp_max': 21.33,
    'pressure': 1011,
    'sea_level': 1011,
    'grnd_level': 1001,
    'humidity': 70,
    'temp_kf': -1.1},
   'weather': [{'id': 802,
     'main': 'Clouds',
     'description': 'scattered clouds',
     'icon': '03d'}],
   'clouds': {'all': 27},
   'wind': {'speed': 2.49, 'deg': 232, 'gust': 2.72},
   'visibility': 10000,
   'pop': 0,
   'sys': {'pod': 'd'},
   'dt_txt': '2023-10-15 12:00:00'},
  {'dt': 1697382000,
   'main': {'temp': 20.19,
    'feels_like': 19.91,
    'temp_min': 20.19,
    'temp_max': 20.44,
    'pressure': 1011,
    'sea_level': 1011,
    'grnd_level': 1001,
    'humidity': 63,
    'temp_kf': -0.25},
   'weather': [{'id': 801,
     'main': 'Clouds',
     'description': 'few clouds',
     'icon': '02d'}],
   'clouds': {'all': 13},
   'wind': {'speed': 2.15, 'deg': 208, 'gust': 2.68},
   'visibilit

In [9]:
weather_json['city']['name']

'Istanbul'

In [10]:
cities_df.iloc[1]['city']

'London'

### Turn dictionary into a Dataframe

In [11]:
# turn dictionary to a dataframe

weather_forecast_df = pd.DataFrame(city_conditions_dict)
weather_forecast_df

,city_id,time,temp_min,temp_max,pop,wind_speed,description,humidity
0,0,2023-10-15 12:00:00,9.40,9.68,0.30,6.18,light rain,66
1,0,2023-10-15 15:00:00,9.75,9.99,0.44,6.03,light rain,64
2,0,2023-10-15 18:00:00,8.45,8.45,0.00,6.20,broken clouds,66
3,0,2023-10-15 21:00:00,7.61,7.61,0.00,5.83,few clouds,71
4,0,2023-10-16 00:00:00,6.49,6.49,0.00,4.96,few clouds,73
...,...,...,...,...,...,...,...,...
155,3,2023-10-19 21:00:00,16.68,16.68,0.00,1.55,clear sky,68
156,3,2023-10-20 00:00:00,16.75,16.75,0.00,2.73,clear sky,72
157,3,2023-10-20 03:00:00,16.48,16.48,0.00,3.91,clear sky,72
158,3,2023-10-20 06:00:00,17.52,17.52,0.00,4.03,clear sky,67


### export to .csv file 

In [12]:
weather_forecast_df.to_csv('weather.csv', index=False)

## Connect to the MySQL database

In [13]:
weather_forecast_df = pd.read_csv('weather.csv')
weather_forecast_df

,city_id,time,temp_min,temp_max,pop,wind_speed,description,humidity
0,0,2023-10-15 12:00:00,9.40,9.68,0.30,6.18,light rain,66
1,0,2023-10-15 15:00:00,9.75,9.99,0.44,6.03,light rain,64
2,0,2023-10-15 18:00:00,8.45,8.45,0.00,6.20,broken clouds,66
3,0,2023-10-15 21:00:00,7.61,7.61,0.00,5.83,few clouds,71
4,0,2023-10-16 00:00:00,6.49,6.49,0.00,4.96,few clouds,73
...,...,...,...,...,...,...,...,...
155,3,2023-10-19 21:00:00,16.68,16.68,0.00,1.55,clear sky,68
156,3,2023-10-20 00:00:00,16.75,16.75,0.00,2.73,clear sky,72
157,3,2023-10-20 03:00:00,16.48,16.48,0.00,3.91,clear sky,72
158,3,2023-10-20 06:00:00,17.52,17.52,0.00,4.03,clear sky,67


In [17]:
host = '127.0.0.1'
schema = 'gans_local' # The name of your database (schema) might be different
user = 'root'
password = 'sql_key'
port = 3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [ ]:
weather_forecast_df.to_sql('weather',con=con,if_exists='append',index=False)

In [19]:
weather_forecast_df
print(weather_forecast_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   city_id      160 non-null    int64  
 1   time         160 non-null    object 
 2   temp_min     160 non-null    float64
 3   temp_max     160 non-null    float64
 4   pop          160 non-null    float64
 5   wind_speed   160 non-null    float64
 6   description  160 non-null    object 
 7   humidity     160 non-null    int64  
dtypes: float64(4), int64(2), object(2)
memory usage: 10.1+ KB
None
